# AI Ethics Hackathon

## NYPD Complaints 1985 - 2020

The problem: **Predictive policing**

If you train a model on biased data (data where systemic discrimination is deeply rooted) then you well get biased predictions that will perpetuate discrimination by law enforcement. The use of predictive policing can also contribute to further accountability by police officers via [tech-washing](https://www.brennancenter.org/our-work/research-reports/predictive-policing-explained).

So is there a place for predictive policing?

----

Well, what is the purpose of predictive policing?

- forecasting crime 

and the purpose of forecasting crime or any sort of policing? 

[Wikipedia](https://en.wikipedia.org/wiki/Police): "The police are a constituted body of persons empowered by a state, with the aim to enforce the law, **to ensure the safety, health and possessions of citizens**, and to prevent crime and civil disorder." 

[Charles Koch Institute](https://www.charleskochinstitute.org/issue-areas/criminal-justice-policing-reform/role-of-police-in-america/): "The purpose of law enforcement in a free society is to promote public safety and uphold the rule of law so that individual liberty may flourish. Trust and **accountability** between law enforcement and the communities they are sworn to protect is essential to advancing these goals." 

- To serve and protect

With the growth of technology so has the awareness of police brutality and the disproportionate harm (often with a lack of police accountability) to Black Americans. 

**In that sense, if the purpose of predictive policing at the highest level is to protect citizens then maybe instead we should determine where (precincts) officers are harming citizens ("hotspots") and whether they will be held accountable.**


In this notebook we will be cleaning and completing EDA on a database of more than 12,000 civilian complaints filed against New York City police officers. https://www.propublica.org/datastore/dataset/civilian-complaints-against-new-york-city-police-officers

In [1]:
# Imports
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 

https://theintercept.com/2020/08/23/nypd-75th-precinct-police-misconduct/


## Cleaning

Let's start by reading in the dataset.

In [2]:
# Read csv as a dataframe
df_raw = pd.read_csv('CCRB-Complaint-Data_202007271729/allegations_202007271729.csv')

# Print the shape of the raw df 
print(f'The dataframe has {df_raw.shape[0]} rows and {df_raw.shape[1]} columns. \n')

# Checking out first 5 rows
df_raw.head()

The dataframe has 33358 rows and 27 columns. 



,unique_mos_id,first_name,last_name,command_now,shield_no,complaint_id,month_received,year_received,month_closed,year_closed,...,mos_age_incident,complainant_ethnicity,complainant_gender,complainant_age_incident,fado_type,allegation,precinct,contact_reason,outcome_description,board_disposition
0,10004,Jonathan,Ruiz,078 PCT,8409,42835,7,2019,5,2020,...,32,Black,Female,38.0,Abuse of Authority,Failure to provide RTKA card,78.0,Report-domestic dispute,No arrest made or summons issued,Substantiated (Command Lvl Instructions)
1,10007,John,Sears,078 PCT,5952,24601,11,2011,8,2012,...,24,Black,Male,26.0,Discourtesy,Action,67.0,Moving violation,Moving violation summons issued,Substantiated (Charges)
2,10007,John,Sears,078 PCT,5952,24601,11,2011,8,2012,...,24,Black,Male,26.0,Offensive Language,Race,67.0,Moving violation,Moving violation summons issued,Substantiated (Charges)
3,10007,John,Sears,078 PCT,5952,26146,7,2012,9,2013,...,25,Black,Male,45.0,Abuse of Authority,Question,67.0,PD suspected C/V of violation/crime - street,No arrest made or summons issued,Substantiated (Charges)
4,10009,Noemi,Sierra,078 PCT,24058,40253,8,2018,2,2019,...,39,NaN,NaN,16.0,Force,Physical force,67.0,Report-dispute,Arrest - other violation/crime,Substantiated (Command Discipline A)


Let's learn more about each column, their datatypes and get an initial feel for any missing data. 

In [3]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33358 entries, 0 to 33357
Data columns (total 27 columns):
unique_mos_id               33358 non-null int64
first_name                  33358 non-null object
last_name                   33358 non-null object
command_now                 33358 non-null object
shield_no                   33358 non-null int64
complaint_id                33358 non-null int64
month_received              33358 non-null int64
year_received               33358 non-null int64
month_closed                33358 non-null int64
year_closed                 33358 non-null int64
command_at_incident         31814 non-null object
rank_abbrev_incident        33358 non-null object
rank_abbrev_now             33358 non-null object
rank_now                    33358 non-null object
rank_incident               33358 non-null object
mos_ethnicity               33358 non-null object
mos_gender                  33358 non-null object
mos_age_incident            33358 non-null int64

Later on we will determine whether 'complainant_age_incident' and 'precinct' should be of float or integer datatypes.

Already we can see that there is missing data that we will have to deal with. Let's see the proportion of missing data for each column.

In [4]:
# Proportion of null values
df_raw.isna().sum()/len(df_raw)

unique_mos_id               0.000000
first_name                  0.000000
last_name                   0.000000
command_now                 0.000000
shield_no                   0.000000
complaint_id                0.000000
month_received              0.000000
year_received               0.000000
month_closed                0.000000
year_closed                 0.000000
command_at_incident         0.046286
rank_abbrev_incident        0.000000
rank_abbrev_now             0.000000
rank_now                    0.000000
rank_incident               0.000000
mos_ethnicity               0.000000
mos_gender                  0.000000
mos_age_incident            0.000000
complainant_ethnicity       0.133821
complainant_gender          0.125757
complainant_age_incident    0.144253
fado_type                   0.000000
allegation                  0.000030
precinct                    0.000719
contact_reason              0.005966
outcome_description         0.001679
board_disposition           0.000000
d

Let's get to it!

In [5]:
# Making a copy of df_raw to clean 
df_clean = df_raw.copy() 

### Let's start with **'outcome_description'**

In [6]:
df_clean['outcome_description'].value_counts(normalize=True)

No arrest made or summons issued        0.385022
Arrest - other violation/crime          0.306168
Summons - disorderly conduct            0.063600
Summons - other violation/crime         0.058255
Arrest - resisting arrest               0.047835
Arrest - disorderly conduct             0.030419
Arrest - assault (against a PO)         0.025584
Moving violation summons issued         0.025194
Arrest - OGA                            0.019488
Other VTL violation summons issued      0.015945
Parking summons issued                  0.008378
Disorderly-Conduct/Arr/Summons          0.004114
Arrest on Other Charge                  0.002432
Traffic Summons Claimed or Issued       0.001772
Juvenile Report                         0.001712
Other Summons Claimed or Issued         0.001141
Assault/Arrested                        0.001021
Resisting Arrest/Arrested               0.000751
Arrest - harrassment (against a PO)     0.000450
Obstruct-Govt-Admin/Arrested            0.000300
Harrassment/Arrested

We can see the most common (mode average) outcome description is that no arrest was made or summons issued. 

We could justify filling the missing cells with this average... However, we could opt to be totally truthful and describe the outcome as it is, which is "`Unknown`."

In [7]:
# Filling missing data with 'Unknown'
df_clean['outcome_description'] = df_clean['outcome_description'].fillna('Unknown')

# Sanity check
print(f"There are {df_clean['outcome_description'].isna().sum()} nulls.")

df_clean['outcome_description'].value_counts(normalize=True)

There are 0 nulls.


No arrest made or summons issued        0.384376
Arrest - other violation/crime          0.305654
Summons - disorderly conduct            0.063493
Summons - other violation/crime         0.058157
Arrest - resisting arrest               0.047755
Arrest - disorderly conduct             0.030368
Arrest - assault (against a PO)         0.025541
Moving violation summons issued         0.025151
Arrest - OGA                            0.019456
Other VTL violation summons issued      0.015918
Parking summons issued                  0.008364
Disorderly-Conduct/Arr/Summons          0.004107
Arrest on Other Charge                  0.002428
Traffic Summons Claimed or Issued       0.001769
Juvenile Report                         0.001709
Unknown                                 0.001679
Other Summons Claimed or Issued         0.001139
Assault/Arrested                        0.001019
Resisting Arrest/Arrested               0.000749
Arrest - harrassment (against a PO)     0.000450
Obstruct-Govt-Admin/

### Dealing with missing data in **'contact_reason'**

We will fill the missing data with "Unknown" again. 

In [8]:
df_clean['contact_reason'].value_counts(normalize=True)

PD suspected C/V of violation/crime - street                  0.303930
Other                                                         0.123767
PD suspected C/V of violation/crime - auto                    0.089900
PD suspected C/V of violation/crime - bldg                    0.076661
Moving violation                                              0.059803
Other violation of VTL                                        0.034380
Report-dispute                                                0.032721
Execution of search warrant                                   0.027534
Report of other crime                                         0.027323
Execution of arrest/bench warrant                             0.020598
Parking violation                                             0.019150
Report-domestic dispute                                       0.016979
C/V intervened on behalf of/observed encounter w/3rd party    0.016285
Report-gun possession/shots fired                             0.014838
PD sus

To tidy this up a bit:


In [9]:
contact_dict = {
    'Others' : 'Other',
    'Aided Cases' : 'Aided case',
    'EDP Aided Cases' : 'EDP aided case',
    'Demonstrations' : 'Demonstration/protest',
    'Traffic accident' : 'Traffic Incidents/Accident/Prk Violation',
    'Report of Disturbance/Noise Complaint' : 'Report-noise/disturbance'
}

df_clean['contact_reason'] = df_clean['contact_reason'].replace(contact_dict)

# Sanity check 
df_clean['contact_reason'].value_counts(normalize=True)

PD suspected C/V of violation/crime - street                  0.303930
Other                                                         0.129467
PD suspected C/V of violation/crime - auto                    0.089900
PD suspected C/V of violation/crime - bldg                    0.076661
Moving violation                                              0.059803
Other violation of VTL                                        0.034380
Report-dispute                                                0.032721
Execution of search warrant                                   0.027534
Report of other crime                                         0.027323
Execution of arrest/bench warrant                             0.020598
Parking violation                                             0.019150
Report-domestic dispute                                       0.016979
C/V intervened on behalf of/observed encounter w/3rd party    0.016285
Report-gun possession/shots fired                             0.014838
Traffi

We could probably spend more time cleaning this up, but we have to move on. 

Let's fill the missing contact reasons with '`Unknown`.'

In [10]:
# Fill na with Unknown
df_clean['contact_reason'] = df_clean['contact_reason'].fillna('Unknown')

# Sanity check 
df_clean['contact_reason'].isna().any()

False

Great we can try and tackle the missing precincts now. 

### Cleaning up the 'precinct' column


In [11]:
# Checking that all are whole numbers
df_clean['precinct'].apply(float.is_integer).all()

False

In [12]:
# Getting rows that returned False as being a whole number/integer
precinct_false_int = df_clean[~df_clean['precinct'].apply(float.is_integer)]

# Getting a proportion of null values
precinct_false_int['precinct'].isna().value_counts(normalize=True)

True    1.0
Name: precinct, dtype: float64

In [13]:
# Display
precinct_false_int[['unique_mos_id','first_name', 'last_name','command_at_incident', 'command_now']]

,unique_mos_id,first_name,last_name,command_at_incident,command_now
6602,18399,Douglas,Lansing,SOD T/U,D-E T/F
6806,18477,John,Acconi,DA QNS,DA QNS
7037,18561,Thomas,Mclaughlin,CENROBB,DB BXHM
9322,19561,Michael,Greaney,WARRSEC,DBQS
12780,21933,Tomasz,Fil,HWY 03,HWY 03
12781,21933,Tomasz,Fil,HWY 03,HWY 03
12782,21933,Tomasz,Fil,HWY 03,HWY 03
15702,23593,Anthony,Quagliariello,MTS DET,JT T/F
15703,23593,Anthony,Quagliariello,MTS DET,JT T/F
15704,23593,Anthony,Quagliariello,MTS DET,JT T/F


MTS = Midtown South Precinct, so MTS will be added to precinct

In [14]:
df_clean.loc[15702:15705, 'precinct'] = "MTS"
df_clean.loc[28692:28693, 'precinct'] = 47
df_clean.loc[29722:29723, 'precinct'] = 52
df_clean.loc[30891, 'precinct'] = 68
df_clean.loc[31956, 'precinct'] = 73

# Sanity check
df_clean[df_clean['precinct'].isna()]

,unique_mos_id,first_name,last_name,command_now,shield_no,complaint_id,month_received,year_received,month_closed,year_closed,...,mos_age_incident,complainant_ethnicity,complainant_gender,complainant_age_incident,fado_type,allegation,precinct,contact_reason,outcome_description,board_disposition
6602,18399,Douglas,Lansing,D-E T/F,6597,6236,3,2003,3,2004,...,35,Black,Male,23.0,Force,Gun Pointed,NaN,Other,Arrest - other violation/crime,Exonerated
6806,18477,John,Acconi,DA QNS,394,43400,10,2019,4,2020,...,49,Unknown,Female,58.0,Discourtesy,Word,NaN,Other,No arrest made or summons issued,Substantiated (Command Lvl Instructions)
7037,18561,Thomas,Mclaughlin,DB BXHM,5097,30178,2,2014,11,2014,...,41,NaN,NaN,NaN,Abuse of Authority,Premises entered and/or searched,NaN,PD suspected C/V of violation/crime - bldg,Arrest - other violation/crime,Unsubstantiated
9322,19561,Michael,Greaney,DBQS,7490,33535,9,2015,3,2016,...,30,NaN,NaN,NaN,Abuse of Authority,Premises entered and/or searched,NaN,Execution of arrest/bench warrant,No arrest made or summons issued,Substantiated (Command Discipline B)
12780,21933,Tomasz,Fil,HWY 03,21392,39202,3,2018,10,2018,...,36,White,Male,40.0,Discourtesy,Word,NaN,Moving violation,Moving violation summons issued,Unsubstantiated
12781,21933,Tomasz,Fil,HWY 03,21392,39202,3,2018,10,2018,...,36,White,Male,40.0,Discourtesy,Word,NaN,Moving violation,Moving violation summons issued,Unsubstantiated
12782,21933,Tomasz,Fil,HWY 03,21392,39202,3,2018,10,2018,...,36,White,Male,40.0,Abuse of Authority,Threat of arrest,NaN,Moving violation,Moving violation summons issued,Unsubstantiated
26077,35924,Jason,Garcia,WARRSEC,2362,33647,10,2015,1,2016,...,43,Black,Female,30.0,Abuse of Authority,Premises entered and/or searched,NaN,Execution of arrest/bench warrant,No arrest made or summons issued,Substantiated (Command Discipline B)
26403,36019,John,Mccrossen,WARRSEC,6353,33535,9,2015,3,2016,...,42,Black,Male,54.0,Abuse of Authority,Refusal to provide name/shield number,NaN,Execution of arrest/bench warrant,No arrest made or summons issued,Substantiated (Command Discipline A)
26404,36019,John,Mccrossen,WARRSEC,6353,33535,9,2015,3,2016,...,42,NaN,NaN,NaN,Abuse of Authority,Premises entered and/or searched,NaN,Execution of arrest/bench warrant,No arrest made or summons issued,Substantiated (Command Discipline B)


In [15]:
df_clean['precinct'] = df_clean['precinct'].fillna('Not Applicable')

# Sanity check 
df_clean['precinct'].isna().any()

False

No 0th precinct. 

In [16]:
for i in df_clean[df_clean['precinct'] == 0].index:
    
    df_clean.loc[i, 'precinct'] = "Not Applicable"

Cleaning up the string

In [17]:
# To string
df_clean['precinct'] = df_clean['precinct'].astype(str)

# Get rid of float remnents
df_clean['precinct'] = df_clean['precinct'].map(lambda x: x.rstrip('.0'))

# Sanity check 
df_clean['precinct'].unique()

array(['78', '67', '79', '77', '81', '73', '9', '75', '12', '1', '25',
       '11', '83', '46', '28', '68', '7', '72', '122', '113', '101', '13',
       '71', '23', '112', '69', '32', '94', '63', '76', '52', '47', '62',
       '42', '84', '115', '109', '5', '88', '14', '103', '107', '104',
       '4', '108', '45', '18', '102', '114', '19', '2', '106', '105',
       '17', '6', '34', '44', '48', '41', '43', '111', '49', '26', '3',
       '33', '61', '123', '121', '66', '24', 'Not Applicable', '22',
       'MTS'], dtype=object)

### Cleaning up the 'complainant_age_incident' column


In [18]:
# Checking that all are whole numbers
df_raw['complainant_age_incident'].apply(float.is_integer).all()

False

Let's take a look at the non-integers: 

In [19]:
# Getting rows that returned False as being a whole number/integer
age_false_int = df_raw[~df_raw['complainant_age_incident'].apply(float.is_integer)]

# Display
age_false_int

,unique_mos_id,first_name,last_name,command_now,shield_no,complaint_id,month_received,year_received,month_closed,year_closed,...,mos_age_incident,complainant_ethnicity,complainant_gender,complainant_age_incident,fado_type,allegation,precinct,contact_reason,outcome_description,board_disposition
68,10039,Willie,Johnson,079 DET,4446,14557,10,2007,3,2009,...,36,NaN,NaN,NaN,Abuse of Authority,Premises entered and/or searched,73.0,Execution of search warrant,No arrest made or summons issued,Exonerated
72,10039,Willie,Johnson,079 DET,4446,18085,4,2009,6,2010,...,37,NaN,NaN,NaN,Abuse of Authority,Gun Drawn,75.0,Other,Arrest - other violation/crime,Exonerated
75,10039,Willie,Johnson,079 DET,4446,18085,4,2009,6,2010,...,37,NaN,NaN,NaN,Abuse of Authority,Premises entered and/or searched,75.0,Other,Arrest - other violation/crime,Substantiated (Charges)
81,10039,Willie,Johnson,079 DET,4446,21823,10,2010,4,2011,...,38,Hispanic,Male,NaN,Discourtesy,Word,75.0,Execution of search warrant,Arrest - other violation/crime,Unsubstantiated
82,10039,Willie,Johnson,079 DET,4446,21823,10,2010,4,2011,...,38,Hispanic,Male,NaN,Abuse of Authority,Threat of force (verbal or physical),75.0,Execution of search warrant,Arrest - other violation/crime,Unsubstantiated
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33276,9796,Steven,Rupnick,077 PCT,27055,38128,10,2017,11,2018,...,27,Unknown,Male,NaN,Abuse of Authority,Interference with recording,77.0,PD suspected C/V of violation/crime - street,Arrest - other violation/crime,Unsubstantiated
33277,9796,Steven,Rupnick,077 PCT,27055,38128,10,2017,11,2018,...,27,Unknown,Male,NaN,Abuse of Authority,Threat of force (verbal or physical),77.0,PD suspected C/V of violation/crime - street,Arrest - other violation/crime,Unsubstantiated
33278,9796,Steven,Rupnick,077 PCT,27055,38128,10,2017,11,2018,...,27,Unknown,Not described,NaN,Abuse of Authority,Threat of force (verbal or physical),77.0,PD suspected C/V of violation/crime - street,Arrest - other violation/crime,Unsubstantiated
33279,9796,Steven,Rupnick,077 PCT,27055,38128,10,2017,11,2018,...,27,Unknown,Not described,NaN,Abuse of Authority,Threat of arrest,77.0,PD suspected C/V of violation/crime - street,Arrest - other violation/crime,Unsubstantiated


From what we can see above, it seems that the entries that were not whole numbers were null values. 

However, let's make sure. 

In [20]:
# Getting a proportion of null values
age_false_int['complainant_age_incident'].isna().value_counts(normalize=True)

True    1.0
Name: complainant_age_incident, dtype: float64

Yes, 100% of the entries that were not whole numbers were null values. Thus, 'complainant_age_incident' should be of `int` datatype. 

However, we will have to deal with the missing values first. 

Let's get a quick overview. 

In [21]:
df_clean['complainant_age_incident'].describe()

count    28546.000000
mean        32.484201
std         28.408963
min      -4301.000000
25%         23.000000
50%         30.000000
75%         41.000000
max        101.000000
Name: complainant_age_incident, dtype: float64

The minimum age doesn't make any sense. Anything less than 0 we will first set to NaN.  

In [22]:
# If complainant_age_incident < 0 then change it to NaN
for i in df_clean[df_clean['complainant_age_incident'] <0]['complainant_age_incident'].index:
    df_clean.loc[i, 'complainant_age_incident'] = np.NaN
    
# Sanity check
df_clean['complainant_age_incident'].describe()

count    28540.000000
mean        32.641906
std         12.206667
min          0.000000
25%         23.000000
50%         30.000000
75%         41.000000
max        101.000000
Name: complainant_age_incident, dtype: float64

Now that we've removed the impossible complainant ages (0 years also isn't ideal, but we'll move on) let's fill the remaining missing data with the median of the column. 

In [23]:
df_clean['complainant_age_incident'].median()

30.0

In [24]:
# Fill null values with the median 
df_clean['complainant_age_incident'] = df_clean['complainant_age_incident'].fillna(df_clean['complainant_age_incident'].median())

# Sanity check that there are no nulls
df_clean['complainant_age_incident'].isna().any()

False

In [25]:
df_clean['complainant_age_incident'].describe()

count    33358.000000
mean        32.260327
std         11.328886
min          0.000000
25%         24.000000
50%         30.000000
75%         38.000000
max        101.000000
Name: complainant_age_incident, dtype: float64

Lastly, we will change the datatype to int. 

In [26]:
# Changing to int datatype
df_clean['complainant_age_incident'] = df_clean['complainant_age_incident'].astype(int)

# Sanity check
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33358 entries, 0 to 33357
Data columns (total 27 columns):
unique_mos_id               33358 non-null int64
first_name                  33358 non-null object
last_name                   33358 non-null object
command_now                 33358 non-null object
shield_no                   33358 non-null int64
complaint_id                33358 non-null int64
month_received              33358 non-null int64
year_received               33358 non-null int64
month_closed                33358 non-null int64
year_closed                 33358 non-null int64
command_at_incident         31814 non-null object
rank_abbrev_incident        33358 non-null object
rank_abbrev_now             33358 non-null object
rank_now                    33358 non-null object
rank_incident               33358 non-null object
mos_ethnicity               33358 non-null object
mos_gender                  33358 non-null object
mos_age_incident            33358 non-null int64

### Dealing with missing data in 'complainant_ethnicity' and 'complainant_gender'

In [27]:
df_clean['complainant_ethnicity'].value_counts()

Black              17114
Hispanic            6424
White               2783
Unknown             1041
Other Race           677
Asian                532
Refused              259
American Indian       64
Name: complainant_ethnicity, dtype: int64

Although we don't know if the ethnicity data is missing because the complainant refused to share it, we do know that it is technically "`Unknown`."

In [28]:
# Fill missing data with Unknown
df_clean['complainant_ethnicity'] = df_clean['complainant_ethnicity'].fillna('Unknown')

# Sanity check 
df_clean['complainant_ethnicity'].isna().any()

False

Now the gender: 

In [29]:
df_clean['complainant_gender'].value_counts()

Male                     24058
Female                    5021
Not described               57
Transwoman (MTF)            20
Transman (FTM)               5
Gender non-conforming        2
Name: complainant_gender, dtype: int64

We will fill missing gender with "not described."

In [30]:
# Missing data to "Not described" group
df_clean['complainant_gender'] = df_clean['complainant_gender'].fillna('Not described')

# Sanity check 
df_clean['complainant_gender'].isna().any()

False

### Now 'command_at_incident'

In [31]:
df_clean[df_clean['command_at_incident'].isna()][['rank_incident','rank_abbrev_incident','precinct']]

,rank_incident,rank_abbrev_incident,precinct
271,Police Officer,POM,73
272,Police Officer,POM,73
340,Police Officer,PO,79
702,Police Officer,POM,83
899,Police Officer,PO,77
...,...,...,...
32606,Police Officer,PO,75
32607,Police Officer,PO,75
32629,Police Officer,PO,75
32936,Police Officer,PO,75


In [32]:
command_nulls = df_clean[df_clean['command_at_incident'].isna()]

to_pct = command_nulls[(command_nulls['rank_incident'] == 'Police Officer') | 
                       (command_nulls['rank_incident'] == 'Sergeant') | 
                       (command_nulls['rank_incident'] == 'Captain') | 
                       (command_nulls['rank_incident'] == 'Lieutenant')].index

# Sanity check 
to_pct

Int64Index([  271,   272,   340,   702,   899,   900,   901,   902,   903,
              904,
            ...
            32601, 32602, 32603, 32604, 32605, 32606, 32607, 32629, 32936,
            32937],
           dtype='int64', length=1504)

In [33]:
for i in to_pct:
    
    if (df_clean.loc[i, 'precinct'] == 'Not Applicable'):
        
        df_clean.loc[i, 'command_at_incident'] = 'Not Applicable'
    
    elif (int(df_clean.loc[i, 'precinct']) < 10):
        
        df_clean.loc[i, 'command_at_incident'] = '00' + str(int(df_clean.loc[i, 'precinct'])) + ' PCT'
    
    elif (int(df_clean.loc[i, 'precinct']) >= 10) & (int(df_clean.loc[i, 'precinct']) < 100):
        
        df_clean.loc[i, 'command_at_incident'] = '0' + str(int(df_clean.loc[i, 'precinct'])) + ' PCT'
    
    elif int(df_clean.loc[i, 'precinct']) >= 100:
        
        df_clean.loc[i, 'command_at_incident'] = str(int(df_clean.loc[i, 'precinct'])) + ' PCT'
        
    else:
        continue
        
# Sanity check
df_clean[df_clean['command_at_incident'].isna()][['rank_incident','rank_abbrev_incident','precinct']]

,rank_incident,rank_abbrev_incident,precinct
6216,Detective,DT3,7
6217,Detective,DT3,7
6218,Detective,DT3,7
6219,Detective,DT3,7
6522,Detective,DET,114
6523,Detective,DT3,114
6524,Detective,DET,114
8016,Detective,DET,23
8017,Detective,DET,23
8569,Detective,DET,7


In [34]:
det = df_clean[df_clean['command_at_incident'].isna()].index

# Sanity check 
det

Int64Index([ 6216,  6217,  6218,  6219,  6522,  6523,  6524,  8016,  8017,
             8569,  8572,  8573,  8574,  8575,  8668,  8669,  8670,  8671,
             8672,  8673,  8733,  8734,  8735,  9373,  9968,  9969,  9970,
            15080, 15081, 15082, 15083, 15634, 15699, 15700, 19791, 19793,
            20769, 20770, 25407, 32386],
           dtype='int64')

In [35]:
for i in det:
    
    if (df_clean.loc[i, 'precinct'] == 'Not Applicable'):
        
        df_clean.loc[i, 'command_at_incident'] = 'Not Applicable'
    
    elif (int(df_clean.loc[i, 'precinct']) < 10):
        
        df_clean.loc[i, 'command_at_incident'] = '00' + str(int(df_clean.loc[i, 'precinct'])) + ' DET'
    
    elif (int(df_clean.loc[i, 'precinct']) >= 10) & (int(df_clean.loc[i, 'precinct']) < 100):
        
        df_clean.loc[i, 'command_at_incident'] = '0' + str(int(df_clean.loc[i, 'precinct'])) + ' DET'
    
    elif int(df_clean.loc[i, 'precinct']) >= 100:
        
        df_clean.loc[i, 'command_at_incident'] = str(int(df_clean.loc[i, 'precinct'])) + ' DET'
        
    else:
        continue
        
# Sanity check
df_clean['command_at_incident'].isna().any()

False

Let's get rid of any duplicate complaints first. 

In [36]:
# Checking whether there are duplicated rows
df_clean.duplicated().any()

True

There are duplicated rows. How many?

In [37]:
# Getting a count of the duplicated rows 
print(f'There are {df_clean.duplicated().sum()} duplicate complaints.')

There are 633 duplicate complaints.


In [38]:
# Drop the duplicate rows from df_clean
df_clean = df_clean.drop_duplicates()

# Sanity check 
df_clean.duplicated().any()

False

Okay, let's take a look at the missing data again. 

In [39]:
df_clean.isna().sum()/len(df_clean)

unique_mos_id               0.000000
first_name                  0.000000
last_name                   0.000000
command_now                 0.000000
shield_no                   0.000000
complaint_id                0.000000
month_received              0.000000
year_received               0.000000
month_closed                0.000000
year_closed                 0.000000
command_at_incident         0.000000
rank_abbrev_incident        0.000000
rank_abbrev_now             0.000000
rank_now                    0.000000
rank_incident               0.000000
mos_ethnicity               0.000000
mos_gender                  0.000000
mos_age_incident            0.000000
complainant_ethnicity       0.000000
complainant_gender          0.000000
complainant_age_incident    0.000000
fado_type                   0.000000
allegation                  0.000031
precinct                    0.000000
contact_reason              0.000000
outcome_description         0.000000
board_disposition           0.000000
d

### Taking a look at `allegation` missing data

In [40]:
df_clean[df_clean['allegation'].isna()]

,unique_mos_id,first_name,last_name,command_now,shield_no,complaint_id,month_received,year_received,month_closed,year_closed,...,mos_age_incident,complainant_ethnicity,complainant_gender,complainant_age_incident,fado_type,allegation,precinct,contact_reason,outcome_description,board_disposition
3982,15188,Peter,Ievolo,120 PCT,1517,860,5,1989,5,1989,...,31,Unknown,Not described,30,Force,NaN,7,Patrol Encounter,No arrest made or summons issued,Unsubstantiated


In [41]:
# Drop the one
df_clean = df_clean.dropna()

# Summary 
df_clean.describe()

,unique_mos_id,shield_no,complaint_id,month_received,year_received,month_closed,year_closed,mos_age_incident,complainant_age_incident
count,32724.000000,32724.000000,32724.000000,32724.000000,32724.000000,32724.000000,32724.000000,32724.000000,32724.000000
mean,18161.045318,6450.167736,23916.864228,6.323310,2010.770260,6.472895,2011.573738,32.356955,32.291621
std,9584.620300,7941.179134,11855.535381,3.364952,5.908341,3.343805,5.950295,6.032781,11.363212
min,2.000000,0.000000,517.000000,1.000000,1985.000000,1.000000,1985.000000,20.000000,0.000000
25%,9654.000000,1101.000000,13799.000000,3.000000,2007.000000,4.000000,2008.000000,28.000000,24.000000
50%,19215.000000,3691.000000,25086.500000,6.000000,2012.000000,6.000000,2013.000000,31.000000,30.000000
75%,25412.000000,7156.500000,34165.000000,9.000000,2015.000000,9.000000,2016.000000,36.000000,38.000000
max,36374.000000,31977.000000,43703.000000,12.000000,2020.000000,12.000000,2020.000000,60.000000,101.000000


Although we don't know if the ethnicity data is missing because the complainant refused to share it, we do know that it is technically "`Unknown`."

## Repetitive Police Officer ranks

In [42]:
rank_dict = {
    'POM' : 'PO',
    'POF' : 'PO',
}

df_clean['rank_abbrev_now'] = df_clean['rank_abbrev_now'].replace(rank_dict)
df_clean['rank_abbrev_incident'] = df_clean['rank_abbrev_incident'].replace(rank_dict)

# Sanity check 
print(df_clean['rank_abbrev_now'].value_counts(normalize=True),'\n')
print(df_clean['rank_abbrev_incident'].value_counts(normalize=True))

PO     0.307847
DT3    0.206943
SGT    0.183138
LT     0.084433
DTS    0.038962
SDS    0.036640
DT2    0.032728
CPT    0.021972
LCD    0.019985
DT1    0.019680
SSA    0.013812
DI     0.012712
INS    0.006295
LSA    0.006265
DC     0.004645
AC     0.002628
CCC    0.000581
COI    0.000428
CD     0.000306
Name: rank_abbrev_now, dtype: float64 

PO     0.674276
SGT    0.171250
DT3    0.081805
LT     0.037954
DTS    0.009870
DT2    0.005928
CPT    0.005409
SDS    0.003759
SSA    0.003148
DI     0.002781
DET    0.001253
INS    0.000825
LSA    0.000703
DT1    0.000581
LCD    0.000397
DC     0.000061
Name: rank_abbrev_incident, dtype: float64


## Engineering a 'months_open' feature

In [43]:
df_clean[['month_received','year_received','month_closed','year_closed']]

,month_received,year_received,month_closed,year_closed
0,7,2019,5,2020
1,11,2011,8,2012
2,11,2011,8,2012
3,7,2012,9,2013
4,8,2018,2,2019
...,...,...,...,...
33353,8,2016,2,2017
33354,8,2016,2,2017
33355,8,2016,2,2017
33356,8,2016,2,2017


In [44]:
df_clean['months_open'] = (12 - df_clean['month_received']) + (df_clean['year_closed']-1 - df_clean['year_received'])*12 + df_clean['month_closed']

# Check it out
df_clean[['month_received','year_received','month_closed','year_closed','months_open']]

,month_received,year_received,month_closed,year_closed,months_open
0,7,2019,5,2020,10
1,11,2011,8,2012,9
2,11,2011,8,2012,9
3,7,2012,9,2013,14
4,8,2018,2,2019,6
...,...,...,...,...,...
33353,8,2016,2,2017,6
33354,8,2016,2,2017,6
33355,8,2016,2,2017,6
33356,8,2016,2,2017,6


In [45]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32724 entries, 0 to 33357
Data columns (total 28 columns):
unique_mos_id               32724 non-null int64
first_name                  32724 non-null object
last_name                   32724 non-null object
command_now                 32724 non-null object
shield_no                   32724 non-null int64
complaint_id                32724 non-null int64
month_received              32724 non-null int64
year_received               32724 non-null int64
month_closed                32724 non-null int64
year_closed                 32724 non-null int64
command_at_incident         32724 non-null object
rank_abbrev_incident        32724 non-null object
rank_abbrev_now             32724 non-null object
rank_now                    32724 non-null object
rank_incident               32724 non-null object
mos_ethnicity               32724 non-null object
mos_gender                  32724 non-null object
mos_age_incident            32724 non-null int64

# NOTE: Officers with rank Lieutenant or above do not have shield numbers!

https://en.wikipedia.org/wiki/New_York_City_Police_Department

In [46]:
df_clean[df_clean['shield_no']==0]['rank_now'].value_counts()

Lieutenant                3622
Captain                    719
Deputy Inspector           416
Chiefs and other ranks     281
Inspector                  206
Name: rank_now, dtype: int64

In [47]:
df_clean.reset_index(drop=True).to_csv('cleaned_nypd_complaints.csv')

In [48]:
pd.read_csv('cleaned_nypd_complaints.csv', index_col=0)

,unique_mos_id,first_name,last_name,command_now,shield_no,complaint_id,month_received,year_received,month_closed,year_closed,...,complainant_ethnicity,complainant_gender,complainant_age_incident,fado_type,allegation,precinct,contact_reason,outcome_description,board_disposition,months_open
0,10004,Jonathan,Ruiz,078 PCT,8409,42835,7,2019,5,2020,...,Black,Female,38,Abuse of Authority,Failure to provide RTKA card,78,Report-domestic dispute,No arrest made or summons issued,Substantiated (Command Lvl Instructions),10
1,10007,John,Sears,078 PCT,5952,24601,11,2011,8,2012,...,Black,Male,26,Discourtesy,Action,67,Moving violation,Moving violation summons issued,Substantiated (Charges),9
2,10007,John,Sears,078 PCT,5952,24601,11,2011,8,2012,...,Black,Male,26,Offensive Language,Race,67,Moving violation,Moving violation summons issued,Substantiated (Charges),9
3,10007,John,Sears,078 PCT,5952,26146,7,2012,9,2013,...,Black,Male,45,Abuse of Authority,Question,67,PD suspected C/V of violation/crime - street,No arrest made or summons issued,Substantiated (Charges),14
4,10009,Noemi,Sierra,078 PCT,24058,40253,8,2018,2,2019,...,Unknown,Not described,16,Force,Physical force,67,Report-dispute,Arrest - other violation/crime,Substantiated (Command Discipline A),6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32719,9992,Tomasz,Pulawski,078 PCT,2642,35671,8,2016,2,2017,...,Asian,Male,21,Discourtesy,Word,66,Moving violation,Moving violation summons issued,Unsubstantiated,6
32720,9992,Tomasz,Pulawski,078 PCT,2642,35671,8,2016,2,2017,...,Asian,Male,21,Abuse of Authority,Interference with recording,66,Moving violation,Moving violation summons issued,Unsubstantiated,6
32721,9992,Tomasz,Pulawski,078 PCT,2642,35671,8,2016,2,2017,...,Asian,Male,21,Abuse of Authority,Search (of person),66,Moving violation,Moving violation summons issued,Substantiated (Formalized Training),6
32722,9992,Tomasz,Pulawski,078 PCT,2642,35671,8,2016,2,2017,...,Asian,Male,21,Abuse of Authority,Vehicle search,66,Moving violation,Moving violation summons issued,Substantiated (Formalized Training),6
